In [ ]:
import pandas as pd
import numpy as np
import re
posts = pd.read_csv('posts.csv.gz', compression='gzip', error_bad_lines=False)

## Contagens

In [2]:
blogs = posts[['posts.csv']].groupby(['posts.csv'])
autores = posts[['authorID']].groupby(['authorID'])
print('Postagens: %i' %len(posts))
print('Blogs: %i' %len(blogs))
print('Autores: %i' %len(autores))

Postagens: 820869
Blogs: 87857
Autores: 95774


## Médias

In [3]:
def titleCount(x): 
    try:
        return len(re.findall(r'\w+', x['title']))
    except:
        return 0
    
def contentCount(x): 
    try:
        return len(re.findall(r'\w+', x['content']))
    except:
        return 0
    
posts['titleCount'] = posts.apply(titleCount,axis=1)
posts['contentCount'] = posts.apply(contentCount,axis=1)

In [4]:
np.mean(posts['titleCount'])

4.732660144310481

In [5]:
np.mean(posts['contentCount'])

267.14916143745228

## Blogs mais densos

In [6]:
blogLen = posts[['posts.csv','contentCount']].groupby(['posts.csv']).agg(['count','mean'])
blogLen['contentCount'].sort_values('mean',ascending=False).head(10)

,count,mean
posts.csv,,
1717983962433433360,10,62653.500000
1881440988280989601,10,31458.200000
1174338327883930016,10,23428.800000
1156787905348184307,9,22866.555556
1636456055145665266,4,21417.250000
1582126559606127684,10,19176.800000
1024539391998378333,10,19084.000000
1244416782248253699,10,18028.000000
1829942138433192009,10,17691.500000


## Posts mais comentados

In [7]:
posts[['title','replies']].sort_values('replies',ascending=False).head(10)

,title,replies
229668,Urgente! Publicado o Edital do Concurso da PME...,1102
683601,Deixe seu Recado no meu Guestbook,695
577222,Sorteio,553
162172,FÓRUM 3 ANO 3 BIMESTRE 2015,528
455884,Recife - PE,509
598118,"Olá, BEM VINDOS !",508
719739,2 - EXPRESSÕES NUMÉRICAS,494
810996,Bate-papo com Atendimento Publicitário,489
97779,Universidade da Flórida descobre cura para o L...,486
5693,Chá de folha de oliveira - Adorei!!!,479


## Posts Auto-Reflexivos

In [8]:
posts = posts.fillna('')
postsReflexivos = posts[posts['content'].str.contains("eu sinto|eu penso|eu acredito")]

print('Posts Reflexivos: %i' %len(postsReflexivos))
print('Total de Posts: %i' %len(posts))
print('Percentual: %f' %round(100 * len(postsReflexivos) / len(posts),3))

Posts Reflexivos: 8428
Total de Posts: 820869
Percentual: 1.000000


## Blogs Auto-Reflexivos

In [9]:
groupReflex = postsReflexivos[['posts.csv','contentCount']].groupby(['posts.csv']).agg(['count','mean'])
groupReflex['contentCount'].sort_values('count',ascending=False).head(10)

,count,mean
posts.csv,,
1717983962433433360,8,77737.750000
118502485184438267,7,562.714286
1450650900147172667,6,247.500000
1314320197865475641,6,8515.000000
1588511966346755811,6,293.500000
1661881252606013579,6,1627.000000
1347353377192780674,5,6499.200000
1874640306506119445,5,642.600000
1634538776961954258,5,1262.400000
